In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageSequence
import numpy as np

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: spam or non-spam
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Data preprocessing using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Updated preprocess_image function
def preprocess_image(img_path):
    img = Image.open(img_path)

    if img.format == 'GIF':
        frames = [np.array(frame.resize((64, 64), Image.ANTIALIAS)) for frame in ImageSequence.Iterator(img)]
        img_array = np.stack(frames)
    else:
        img = img.convert('RGB')
        img = img.resize((64, 64), Image.ANTIALIAS)
        img_array = np.expand_dims(np.array(img), axis=0)

    img_array = img_array / 255.0
    return img_array

train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'  # Binary labels for spam or non-spam
)

validation_generator = test_datagen.flow_from_directory(
    'Dataset/Test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Save the model for future use
model.save('image_spam_detection_model.h5')

Found 191 images belonging to 3 classes.
Found 61 images belonging to 2 classes.
Epoch 1/10
6/6 [==============================] - 7s 634ms/step - loss: -1.5902 - accuracy: 0.7016 - val_loss: 48.8167 - val_accuracy: 0.0492
Epoch 2/10
6/6 [==============================] - 3s 458ms/step - loss: -38.4357 - accuracy: 0.8482 - val_loss: 512.9125 - val_accuracy: 0.0492
Epoch 3/10
6/6 [==============================] - 3s 456ms/step - loss: -240.5853 - accuracy: 0.8482 - val_loss: 2730.9966 - val_accuracy: 0.0492
Epoch 4/10
6/6 [==============================] - 3s 461ms/step - loss: -1062.7694 - accuracy: 0.8482 - val_loss: 10892.8545 - val_accuracy: 0.0492
Epoch 5/10
6/6 [==============================] - 3s 431ms/step - loss: -3778.9800 - accuracy: 0.8482 - val_loss: 34361.9609 - val_accuracy: 0.0492
Epoch 6/10
6/6 [==============================] - 3s 441ms/step - loss: -11557.1650 - accuracy: 0.8482 - val_loss: 92995.4453 - val_accuracy: 0.0492
Epoch 7/10
6/6 [==========================

C:\Users\DELL\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

# Load the trained model
loaded_model = load_model('image_spam_detection_model.h5')
# Define the target size
target_size = (64, 64)

# Load a sample image for testing
sample_image_path = '30.jpg'  # Replace with the path to your sample image
sample_img = Image.open(sample_image_path)

# Preprocess the sample image
def preprocess_image(img):
    img = img.convert('RGB')
    img = img.resize(target_size, Image.LANCZOS)
    img_array = np.expand_dims(np.array(img), axis=0)
    img_array = img_array / 255.0
    return img_array

# Make predictions
preprocessed_sample = preprocess_image(sample_img)
prediction = loaded_model.predict(preprocessed_sample)

# Get the class label (spam or non-spam)
predicted_class = int(round(prediction[0][0]))

# Get the class name
class_name = 'Spam' if predicted_class == 1 else 'Non-Spam'

# Print the prediction result
print(f'The model predicts that the image is: {class_name}')

C:\Users\DELL\AppData\Local\Temp\ipykernel_16200\3056900746.py:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


1/1 [==============================] - 1s 855ms/step
The model predicts that the image is: Spam
